In [115]:
import numpy as np
import pandas as pd
import pathlib
import os
from decimal import *
getcontext().prec = 5

df_probabilities = {
	6: pd.read_csv('vars\\prob_06.csv', header=None),
	12: pd.read_csv('vars\\prob_12.csv', header=None)
}

df_tables = {
	6: pd.read_csv('vars\\table_06.csv', header=None),
	12: pd.read_csv('vars\\table_12.csv', header=None)
}

lengths = {
	6: {
		'text_count': len(df_tables[6].columns),
		'key_count': len(df_tables[6])
	},
	12: {
		'text_count': len(df_tables[12].columns),
		'key_count': len(df_tables[12])
	}
}


In [116]:
def get_cipher_text_unconditional_probability(variant, cipher_text):
	df_table = df_tables[variant]
	df_prob = df_probabilities[variant]
	
	probability = 0.0

	for plain_text_index in range(lengths[variant]['text_count']):
		for key_text_index in range(lengths[variant]['key_count']):
			if (df_table[plain_text_index][key_text_index] == cipher_text):
				plain_text_probability = df_prob[plain_text_index][0]
				key_probability = df_prob[key_text_index][1]

				probability += plain_text_probability * key_probability
	
	return probability

df_cipher_texts_unconditional_prob = {
	6: np.array([get_cipher_text_unconditional_probability(6, i) for i in range(lengths[6]['text_count'])]),
	12: np.array([get_cipher_text_unconditional_probability(12, i) for i in range(lengths[12]['text_count'])])
}


In [117]:
def cipher_and_plain_texts_common_probability(variant, cipher_text, plain_text):
	df_table = df_tables[variant]
	df_prob = df_probabilities[variant]
	
	probability = 0.0

	for key_index in range(lengths[variant]['key_count']):
		if (df_table[plain_text][key_index] == cipher_text):
			plain_text_probability = df_prob[plain_text][0]
			key_probability = df_prob[key_index][1]

			probability += plain_text_probability * key_probability
	
	return probability

df_cipher_texts_common_prob = {
	6: np.array([[cipher_and_plain_texts_common_probability(6, i, j) for i in range(lengths[6]['text_count'])] for j in range(lengths[6]['text_count'])]),
	12: np.array([[cipher_and_plain_texts_common_probability(12, i, j) for i in range(lengths[12]['text_count'])] for j in range(lengths[12]['text_count'])])
}


In [120]:
def cipher_and_plain_texts_conditional_probability(variant):
    return np.array([
			[df_cipher_texts_common_prob[variant][i][j] / df_cipher_texts_unconditional_prob[variant][i] for i in range(lengths[6]['text_count'])
		] for j in range(lengths[6]['text_count'])])

df_cipher_texts_common_prob = {
	6: cipher_and_plain_texts_conditional_probability(6),
	12: cipher_and_plain_texts_conditional_probability(12)
}


array([[0.        , 0.04      , 0.        , 0.03333333, 0.        ,
        0.03333333, 0.        , 0.23333333, 0.04      , 0.04      ,
        0.03333333, 0.05      , 0.03333333, 0.06666667, 0.01818182,
        0.03333333, 0.04      , 0.        , 0.06666667, 0.06666667],
       [0.        , 0.        , 0.2       , 0.06666667, 0.        ,
        0.06666667, 0.025     , 0.06666667, 0.08      , 0.04      ,
        0.        , 0.        , 0.        , 0.        , 0.01818182,
        0.13333333, 0.08      , 0.025     , 0.        , 0.03333333],
       [0.24      , 0.04      , 0.        , 0.23333333, 0.04      ,
        0.        , 0.        , 0.06666667, 0.        , 0.04      ,
        0.        , 0.025     , 0.1       , 0.1       , 0.01818182,
        0.06666667, 0.        , 0.        , 0.03333333, 0.03333333],
       [0.24      , 0.        , 0.13333333, 0.        , 0.04      ,
        0.06666667, 0.        , 0.06666667, 0.04      , 0.04      ,
        0.03333333, 0.        , 0.        , 0